In [1]:
import torch
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
from torch import nn
import matplotlib.pyplot as plt
from utils import load_nerl_data, get_normalized_adj, get_Laplace, calculate_random_walk_matrix,test_error_cap,test_error
import random
import copy
import scipy
from scipy.io import loadmat
import math
import pandas as pd
from basic_structure import D_GCN, C_GCN, K_GCN,IGNNK
from basic_process import *

2023-12-03 15:26:26.292118: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-03 15:26:26.316336: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-03 15:26:26.655551: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
n_o_n_m = 20 #sampled space dimension

h = 24 #sampled time dimension

z = 100 #hidden dimension for graph convolution

K = 1 #If using diffusion convolution, the actual diffusion convolution step is K+1

n_m = 20 #number of mask node during training

N_u = 50 #target locations, N_u locations will be deleted from the training data

Max_episode = 750 #max training episode

learning_rate = 0.0001 #the learning_rate for Adam optimizer

batch_size = 8

E_maxvalue=80

STmodel = IGNNK(h, z, K)
STmodel = torch.load('model/nrel_ignnk_sigmaA_cap_20210621.pth')

In [18]:
missing_columns

array([ 0, 34, 38, 49, 50, 55, 56, 73, 81, 83])

In [19]:
NDBC_ID

,ID
0,41001
1,41008
2,41012
3,41013
4,41024
...,...
98,mdrm1
99,mism1
100,mkgm4
101,mlrf1


In [16]:
X_raw=np.load('data/NDBC/all.npy')
#Raw data and raw adjacency matrix without mapminmax
X_raw=np.load('data/NDBC/all.npy')
# X_raw=normalize_3d_array(X_raw)
Station_info=pd.read_csv('data/NDBC/Station_info.csv')
NDBC_lat=pd.DataFrame(Station_info.iloc[:,1])
NDBC_long=pd.DataFrame(Station_info.iloc[:,3])
NDBC_ID=pd.DataFrame(Station_info.iloc[:,0])
Adj_dist=adj_dist(NDBC_lat,NDBC_long)
Station_info=pd.read_csv('data/NDBC/Station_info.csv')
NDBC_lat=pd.DataFrame(Station_info.iloc[:,1])
NDBC_long=pd.DataFrame(Station_info.iloc[:,3])
NDBC_ID=pd.DataFrame(Station_info.iloc[:,0])
Adj_dist=adj_dist(NDBC_lat,NDBC_long)
j=5#J-th colum, start from 5 as the first feature.
X_raw_0=X_raw[:,j,:]#GET the first feature
print(X_raw_0.shape)
# Count the num of missing values in each column
missing_counts_per_column = np.sum(np.isnan(X_raw_0), axis=0)
# print results
print("Incomplete data number in each column：", missing_counts_per_column)

#Get the index if the value is not zero
# Find the columns where missing values exist.
columns_with_missing_data = np.any(np.isnan(X_raw_0), axis=0)

# Get the column numbers when missing value exist.
missing_columns = np.where(columns_with_missing_data)[0]

# Print results
print("The column numbers with missing values are：", missing_columns)

#Get new data after deletion.
#Delete those columns(Stations) if there is not any features.
result = np.delete(X_raw_0, missing_columns, axis=1)
result = (result - np.min(result)) / (np.max(result) - np.min(result))
#Get new adjacency matrix after deletion.
NDBC_long=NDBC_long.transpose()
NDBC_long_aft=NDBC_long.drop(columns=missing_columns)
# Reconstruct the index in the DataFrame and get the longitude after deletion.
NDBC_long_aft = NDBC_long_aft.reset_index(drop=True)
print(NDBC_long_aft.shape)

NDBC_lat=NDBC_lat.transpose()
NDBC_lat_aft=NDBC_lat.drop(columns=missing_columns)
# Reconstruct the index in the DataFrame and get the latitude after deletion.
NDBC_lat_aft = NDBC_lat_aft.reset_index(drop=True)
print(NDBC_lat_aft.shape)
#GEt the new ADJ matrix.
Adj_dist=adj_dist(NDBC_lat_aft.transpose(),NDBC_long_aft.transpose())


capacities=np.ones((Adj_dist.shape[0], 1))
capacities=capacities.flatten()
split_line1 = int(result.shape[0] * 0.7)
training_set = result[:split_line1, :]
test_set = result[split_line1:, :]  
rand = np.random.RandomState(0) # Fixed random output, just an example when seed = 0.
unknow_set = rand.choice(list(range(0,result.shape[1])),N_u,replace=False)#Stations to be predicted
unknow_set = set(unknow_set)
full_set = set(range(0,result.shape[1]))
know_set = full_set - unknow_set
training_set_s = training_set[:, list(know_set)]   # get the training data in the sample time period
A_s = Adj_dist[:, list(know_set)][list(know_set), :] 
MAE_t, RMSE_t, R2_t, nrel_ignnk_res  = test_error_cap(STmodel, unknow_set, full_set,test_set, Adj_dist,h,capacities)
#MAE_t, RMSE_t, R2_t, nrel_ignnk_res  = test_error(STmodel, unknow_set, test_set, A_s,E_maxvalue, True)
print('Best model in the',j, '-th column is', MAE_t, RMSE_t, R2_t)

(8784, 103)
Incomplete data number in each column： [8784    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0 8784    0    0    0 8784    0    0    0
    0    0    0    0    0    0    0 8784 8784    0    0    0    0 8784
 8784    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0 8784    0    0    0    0    0    0    0 8784    0 8784
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]
The column numbers with missing values are： [ 0 34 38 49 50 55 56 73 81 83]
(1, 93)
(1, 93)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (93x72 and 48x100)

In [6]:
import folium
#Visualization of all points in google map.
Station_num=Station_info[:,0]
# 创建地图对象
mymap = folium.Map(location=[0, 0], zoom_start=2)  # 初始位置和缩放级别

# 添加经纬度坐标点
coordinates = [
    (37.7749, -122.4194),  # 旧金山
    (40.7128, -74.0060),   # 纽约
    (51.5074, -0.1278),    # 伦敦
    (35.6895, 139.6917),   # 东京
]

# 在地图上标记每个坐标点，并显示编号
for i, coord in enumerate(coordinates, start=1):
    folium.Marker(location=coord, popup=f"Point {i}: {coord}").add_to(mymap)

# 在Jupyter Notebook中显示地图
mymap

In [10]:
NDBC_lat
NDBC_long
NDBC_ID

In [14]:
import pandas as pd
import folium
#Step1: Show all of stations.


# 读取CSV文件
csv_file_path = 'data/NDBC/Station_info.csv'  # 替换为你的CSV文件路径
df = pd.read_csv(csv_file_path)

# 创建地图对象
mymap = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=10)  # 使用平均经纬度作为初始位置

# 在地图上标记每个站点
for index, row in df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"Station {row['ID']}",
    ).add_to(mymap)

# 在Jupyter Notebook中显示地图
mymap


In [15]:
#Step 2: Subgraph generation. Generate 19 neighbors for each node.
import numpy as np

group_num=19#Hyperparameter, it can be any numbers.
result
NDBC_lat_aft
NDBC_long_aft
MI_store=[]
for i in range(result.shape[1]):
    for j in range(result.shape[1]):
        MI_store.append(calc_MI(result[:,i],result[:,j],10))
MI_store=np.array(MI_store).reshape(result.shape[1],result.shape[1])

# 创建一个大小为(93, 93)的随机数组，你可以替换为你的实际数组
a = MI_store

# 对每一行的所有列元素进行排序，并返回最大的19个元素及其列号
max_elements_and_indices = []
for i in range(a.shape[0]):
    row = a[i, :]
    sorted_indices = np.argsort(row)
    max_indices = sorted_indices[-19:][::-1]  # 获取最大的19个元素的列号
    max_elements_and_indices.append([(row[j], j) for j in max_indices])

# 输出结果
for i, row_data in enumerate(max_elements_and_indices):
    print(f"Row {i}: {row_data}")


Row 0: [(3.253426176036873, 0), (0.8272601132894586, 1), (0.6901375697065202, 2), (0.4673606483958732, 4), (0.45757637448269683, 79), (0.4363240338301644, 5), (0.4190235175463517, 20), (0.37428008133733837, 3), (0.37263024544485734, 77), (0.33215728219304985, 21), (0.3299090634675217, 17), (0.3150689617867082, 83), (0.29292039446620866, 85), (0.22793155006306698, 78), (0.22350595260037576, 87), (0.2084899516681764, 91), (0.19725693559810153, 12), (0.16436438822998234, 15), (0.1621107642515911, 31)]
Row 1: [(3.2583097165163166, 1), (0.8272601132894577, 0), (0.5324618722588088, 20), (0.52600823699178, 2), (0.4273072620362397, 85), (0.4023069759074378, 21), (0.39111917114037187, 77), (0.34043212687770286, 4), (0.33742367205695434, 79), (0.3283135513911546, 17), (0.32209271522604777, 5), (0.3192857717713018, 91), (0.3012243820649747, 87), (0.2950417871731199, 83), (0.25018137359531334, 3), (0.2344515509245717, 11), (0.23100847367209631, 15), (0.22954525563754213, 12), (0.22306577157731855,

In [17]:
import numpy as np

# 假设 a 是一个 numpy 数组
a = MI_store

# 对每一行的所有列元素进行排序，并返回最大的19个元素及其列号（不与行号一致）
max_elements_and_indices = []
for i in range(a.shape[0]):
    row = a[i, :]
    sorted_indices = np.argsort(row)
    max_indices = sorted_indices[-19:][::-1]  # 获取最大的19个元素的列号
    # 排除与行号相同的列号
    max_indices = [index for index in max_indices if index != i]
    max_elements_and_indices.append([(row[j], j) for j in max_indices])

# 输出结果
for i, row_data in enumerate(max_elements_and_indices):
    print(f"Row {i}: {row_data}")

Row 0: [(0.8272601132894586, 1), (0.6901375697065202, 2), (0.4673606483958732, 4), (0.45757637448269683, 79), (0.4363240338301644, 5), (0.4190235175463517, 20), (0.37428008133733837, 3), (0.37263024544485734, 77), (0.33215728219304985, 21), (0.3299090634675217, 17), (0.3150689617867082, 83), (0.29292039446620866, 85), (0.22793155006306698, 78), (0.22350595260037576, 87), (0.2084899516681764, 91), (0.19725693559810153, 12), (0.16436438822998234, 15), (0.1621107642515911, 31)]
Row 1: [(0.8272601132894577, 0), (0.5324618722588088, 20), (0.52600823699178, 2), (0.4273072620362397, 85), (0.4023069759074378, 21), (0.39111917114037187, 77), (0.34043212687770286, 4), (0.33742367205695434, 79), (0.3283135513911546, 17), (0.32209271522604777, 5), (0.3192857717713018, 91), (0.3012243820649747, 87), (0.2950417871731199, 83), (0.25018137359531334, 3), (0.2344515509245717, 11), (0.23100847367209631, 15), (0.22954525563754213, 12), (0.22306577157731855, 78)]
Row 2: [(1.0421298348636876, 4), (0.8897537